## Mutation operator based on Oladayo and Mallipeddi - adaptive ensemble ES

In [9]:
import os
import os

# Change to the correct directory where the EvoMan-Generalist folder is located
os.chdir('/Users/christophlaute/Evolutionary/Group-Assignment-Evolutionary-Computing/EvoMan-Generalist')

# Verify the current working directory
print("Current working directory:", os.getcwd())

print("Current working directory:", os.getcwd())

import numpy as np
import random
from evoman.environment import Environment
from demo_controller import player_controller
import os
n_hidden_neurons=10
#n_weights = (env.get_num_sensors() + 1) * n_hidden_neurons + (n_hidden_neurons + 1) * 5
n_weights=265
experiment_name = 'generalist1'
if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

Current working directory: /Users/christophlaute/Evolutionary/Group-Assignment-Evolutionary-Computing/EvoMan-Generalist
Current working directory: /Users/christophlaute/Evolutionary/Group-Assignment-Evolutionary-Computing/EvoMan-Generalist


: 

In [97]:
import numpy as np
import random
def initialize_population(population_size, n_vars):
    population=[]
    individual_mutation_rates=[]
    for i in range(population_size):
        population.append(np.random.uniform(-1, 1, n_vars)) 
    return np.array(population)

def simulation(ind,enemies=[2,4,6]): # just for debugging purpose set enemies as 2, 4, 6
    ''' simulates the environment and evaluates the fitness score against group of enemies
    returns fitness_score: list of fitness scores against each enemy and also average fitness too
    basically fitness_score[0]= fitness score upon enemies[0] and so on 
    and fitness_score[4] = mean(fitness_score) - standard deviation(fitness_score)'''

    individual_scores=[] 
    for i in range(len(enemies)):
      env = Environment(experiment_name=experiment_name,
                  enemies=[enemies[i]],
                  playermode="ai",
                  player_controller=player_controller(n_hidden_neurons),
                  enemymode="static",
                  level=2,
                  speed="fastest",
                  visuals=False)
      f,p,e,t = env.play(pcont=ind)
      individual_scores.append(f)
    
    avg_fitness=env.cons_multi(np.array(individual_scores))
    individual_scores.append(avg_fitness)
    return individual_scores

def evaluation(population, enemies:list):
   '''calls the simulation function for each individual in population 
      returns np array of fitness scores for whole population'''
   
   fitness_scores=[]
   for ind in population:
      individual_scores = simulation(ind,enemies)
      fitness_scores.append(individual_scores)

   return np.array(fitness_scores)

def adaptive_ensemble_mutation(population,p_gaussian,nr_gen,p_mutation,enemies):
    """mutation operator that applies gaussian mutation and cauchy distribution adaptively"""
    #initialize variables
    gaussian_rows = int(p_gaussian*population.shape[0])
    gaussian_indx = np.random.choice(population.shape[0],gaussian_rows,replace=False)
    gaussian_pop = population[gaussian_indx]
    cauchy_pop = np.array([individual for individual in population.tolist() if individual not in gaussian_pop.tolist()])
    success_counter=0

    #mutation per operator_subgroup
    for index,g_ind in  enumerate(gaussian_pop):
        for gene in range(len(g_ind)):
            if p_mutation>random.uniform(0,1):
                gaussian_pop[index][gene]+=np.random.normal(0,1) 
    for index,c_ind in enumerate(cauchy_pop):
        for gene in range(len(c_ind)):
            if p_mutation>random.uniform(0,1):                     #still have to debug
                cauchy_pop[index][gene]+=1+(np.arctan(np.random.standard_cauchy())/np.pi)

    #form new population and evaluate top percent 
    population = np.concatenate((gaussian_pop,cauchy_pop))
    if nr_gen > 10:
        fitness_list = evaluation(population,enemies)
        fitness_pop=fitness_list[-1]
        sorted_fitness_ind = sorted(range(len(fitness_pop)), key=lambda x:fitness_pop[x], reverse=True)
        elites = population[sorted_fitness_ind[:int(0.2*population.shape[0])]]
        for element in elites:
            if element in gaussian_pop:
                success_counter+=1
        p_gaussian = success_counter/len(elites)

    return population,p_gaussian

def get_max_range(population):
    max_range=-1
    for index,individual in enumerate(population):
        for other in population[index+1:]:
            range = abs(np.linalg.norm(individual-other))
            if range>max_range:
                max_range=range
    return max_range
        


#fitness sharing
def shared_fitness_function(population,fitness_values,current_gen,max_gen):
    """function that penalizes fitness if individual is too similar"""
    k=5/max_gen
    s_fit_values=[]
    max_range = get_max_range(population)
    initial_threshold = max_range*0.2
    final_threshold = max_range*0.03
    threshold = final_threshold + (initial_threshold-final_threshold)*np.exp(-k*current_gen)
    
   #loop over individual in population and the possible pairs
    for value in range(population.shape[0]):
        s_fit_pen=0
        for other in range(population.shape[0]):
            if other!=value:
                
                #calculates euclidean distance between individual and candidate for niche
                euc=np.linalg.norm(population[value]-population[other])
                if euc<threshold:
                    
                    #sums penalisation
                    s_fit_pen+=(1-(euc/threshold))
        
        #calculates new value
        if s_fit_pen>0:
            s_fit_value=fitness_values[value]/s_fit_pen
            s_fit_values.append(s_fit_value)
        else:
            s_fit_values.append(fitness_values[value])
    return s_fit_values

def get_diversity(population):
    """calculates how diverse each individual is"""
    mean_ind = np.mean(population,axis=0)
    diversity_list=[]
    for individual in population:
        diversity = abs(np.linalg.norm(individual-mean_ind))
        diversity_list.append(diversity)
    return diversity_list

def two_archives_survival(parents,children,p_fitness,enemies,current_gen,max_gen):
    """function that performs survivor selection proportional to p_fitness based on fitness and otherwise based on diversity"""
    
    survivors=[]
    ind=0
    
    #concatenates parents and children into one array, calculates fitness and diversity
    print(f"shape of parents ={parents.shape}, shape of children = {children.shape}")
    total_pop = np.concatenate((parents,children))
    diversity_p = get_diversity(total_pop)
    total_pop_fit = evaluation(total_pop,enemies)
    total_shared = np.array(shared_fitness_function(total_pop,total_pop_fit,current_gen,max_gen))
    
    #sorts the fitnesses and diversity and returns indeces
    fitness_sorted = np.argsort(total_shared[:, -1])[::-1]
    print(f"fitness_sorted.shape={fitness_sorted.shape}")
    diversity_sorted = np.argsort(diversity_p)[::-1]
    print(f"Initial population shape: {total_pop.shape}")
    print(f"Individual shape after evaluation: {total_pop[fitness_sorted[ind]].shape}")
    #loop that chooses survivors proportionally to p_fitness from fitness or else from diversity
    while len(survivors)<parents.shape[0]:
        if np.random.uniform(0,1)<p_fitness and total_pop[fitness_sorted[ind]].tolist() not in [s.tolist() for s in survivors]:
            print(f"Added to survivors from fitness, shape: {total_pop[fitness_sorted[ind]].shape}")
            survivors.append(total_pop[fitness_sorted[ind]])
        elif total_pop[diversity_sorted[ind]].tolist() not in [s.tolist() for s in survivors]:
                print(f"Added to survivors from diversity, shape: {total_pop[diversity_sorted[ind]].shape}")
                survivors.append(total_pop[diversity_sorted[ind]])
        else:
            if np.random.uniform(0,1)<0.5:
                survivors.append(total_pop[fitness_sorted[ind]])
                print(f"Added to survivors, shape: {total_pop[fitness_sorted[ind]].shape}")
            else:
                survivors.append(total_pop[diversity_sorted[ind]])   
                print(f"Added to survivors, shape: {total_pop[diversity_sorted[ind]].shape}")
        ind+=1
    
    return np.array(survivors)

        

    


In [99]:
#test
parents=initialize_population(100,265)
fitness_values = evaluation(population, [1,2,3])
nr_gen=20
p_mutation=0.3
p_gaussian=0.8
children,p_gaussian = adaptive_ensemble_mutation(population,p_gaussian,nr_gen,p_mutation,[1, 2, 3])


survivors=two_archives_survival(parents,children,0.8,[1,2,3],20,100)
print(survivors.shape)


MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: 

In [38]:
i

Mean array per value: [4. 5. 6.]
